In [1]:
import os
import glob
import pickle
import warnings
warnings.filterwarnings("ignore")
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
plt.ion()
from datetime import datetime
from Toolshed import Download, Toolbox, VegetationLine, Plotting, PlottingSeaborn, Transects
import ee
import geopandas as gpd
import geemap

ee.Initialize()

In [2]:
# Define AOI using coordinates of a rectangle
# The points represent the corners of a bounding box that go around your site
sitename = 'TESTING_A'#'SITENAME'

# Date range
dates = ['2022-01-01', '2022-03-30'] # ['2021-05-01', '2021-07-02']

# Satellite missions
# Input a list of containing any/all of 'L5', 'L7', 'L8', 'L9', 'S2', 'PSScene4Band'
# L5: 1984-2013; L7: 1999-2017 (SLC error from 2003); L8: 2013-present; S2: 2014-present; L9: 2021-present
sat_list = ['L8'] #['L5','L8','S2']

# Cloud threshold for screening out cloudy imagery (0.5 or 50% recommended)
cloud_thresh = 0.5

# Extract shoreline (wet-dry boundary) as well as veg edge
wetdry = False#True

# Reference shoreline/veg line shapefile name (should be stored in a folder called referenceLines in Data)
# Line should be ONE CONTINUOUS linestring along the shore, stored as a shapefile in WGS84 coord system
referenceLineShp = 'EXAMPLE_refLineA.shp' #'SITENAME_refLine.shp'
# Maximum amount in metres by which to buffer the reference line for capturing veg edges within
max_dist_ref = 150

In [3]:
# Directory where the data will be stored
filepath = Toolbox.CreateFileStructure(sitename, sat_list)
# Return AOI from reference line bounding box and save AOI folium map HTML in sitename directory
referenceLinePath = os.path.join(filepath, 'referenceLines', referenceLineShp)
print(referenceLinePath)
referenceLineDF = gpd.read_file(referenceLinePath)
#print(tabulate(referenceLineDF.head(), 'grid'))
#print(referenceLineDF.head())
polygon, point, lonmin, lonmax, latmin, latmax = Toolbox.AOIfromLine(referenceLinePath, max_dist_ref, sitename)

# It's recommended to convert the polygon to the smallest rectangle (sides parallel to coordinate axes)       
polygon = Toolbox.smallest_rectangle(polygon)

if len(dates)>2:
    daterange='no'
else:
    daterange='yes'
years = list(Toolbox.daterange(datetime.strptime(dates[0],'%Y-%m-%d'), datetime.strptime(dates[-1],'%Y-%m-%d')))

# Put all the inputs into a dictionary
inputs = {'polygon': polygon, 'dates': dates, 'daterange':daterange, 'sat_list': sat_list, 'sitename': sitename, 'filepath':filepath}

Sat = Toolbox.image_retrieval(inputs)
metadata = Toolbox.metadata_collection(inputs, Sat)
inputs = Download.check_images_available(inputs)

#BasePath = 'Data/' + sitename + '/lines'
BasePath = 'Data/' + sitename + '/linesV'

if os.path.isdir(BasePath) is False:
    os.mkdir(BasePath)

projection_epsg, _ = Toolbox.FindUTM(polygon[0][0][1],polygon[0][0][0])
print(projection_epsg,'proj')
settings = {
    # general parameters:
    'cloud_thresh': cloud_thresh,        # threshold on maximum cloud cover
    'output_epsg': projection_epsg,     # epsg code of spatial reference system desired for the output   
    'wetdry': wetdry,              # extract wet-dry boundary as well as veg
    # quality control:
    'check_detection': True,    # if True, shows each shoreline detection to the user for validation
    'adjust_detection': False,  # if True, allows user to adjust the postion of each shoreline by changing the threhold
    'save_figure': True,        # if True, saves a figure showing the mapped shoreline for each image
    # [ONLY FOR ADVANCED USERS] shoreline detection parameters:
    'min_beach_area': 200,     # minimum area (in metres^2) for an object to be labelled as a beach
    'buffer_size': 250,         # radius (in metres) for buffer around sandy pixels considered in the shoreline detection
    'min_length_sl': 500,       # minimum length (in metres) of shoreline perimeter to be valid
    'cloud_mask_issue': False,  # switch this parameter to True if sand pixels are masked (in black) on many images  
    # add the inputs defined previously
    'inputs': inputs,
    'projection_epsg': projection_epsg,
    'year_list': years
}

referenceLinePath = os.path.join(inputs['filepath'], 'referenceLines', referenceLineShp)
referenceLine, ref_epsg = Toolbox.ProcessRefline(referenceLinePath,settings)
#print(f'{referenceLinePath}\n{referenceLine}\n{ref_epsg}')

settings['reference_shoreline'] = referenceLine
settings['ref_epsg'] = ref_epsg
# Distance to buffer reference line by (this is in metres)
settings['max_dist_ref'] = max_dist_ref

settings['reference_coreg_im'] = None # leave as None if no coregistration is to be performed

C:\Users\charl\Downloads\DIX\COASTGUARD-master (1)\COASTGUARD-master\Data\referenceLines\EXAMPLE_refLineA.shp
retrieving image metadata...
Metadata already exists and was loaded
Images available between 2022-01-01 and 2022-03-30:
- In Landsat Tier 1 & Sentinel-2 Level-1C:
  L8: 8 images
  Total: 8 images
- In Landsat Tier 2:
  L8: 0 images
  Total: 0 images
32630 proj


In [ ]:
output, output_latlon, output_proj = VegetationLine.extract_veglines(metadata, settings, polygon, dates)

C:\Users\charl\Downloads\DIX\COASTGUARD-master (1)\COASTGUARD-master\Data\TESTING_A
running Toolbox.ResumeVegline...
C:\Users\charl\Downloads\DIX\COASTGUARD-master (1)\COASTGUARD-master\Data\TESTING_A\jpg_files\detection
Mapping veglines:
loop through satellite list
satname
['2022-01-13', '2022-01-29', '2022-03-18', '2022-04-03', '2022-04-19', '2022-05-05', '2022-05-21', '2022-06-06', '2022-06-22', '2022-01-04', '2022-01-20', '2022-02-21', '2022-03-25', '2022-04-10', '2022-04-26', '2022-05-12', '2022-05-28', '2022-06-29']
L8 selected with 15 MLPClassifier_Veg_L5L8S2.pkl
load in trained classifier pkl file
 loop through the images
L8:   5.556 %  # preprocess image (cloud mask + pansharpening/downsampling)
Image_Processing.preprocess_single
 
saving LANDSAT/LC08/C02/T1_TOA/LC08_203024_20220113
# classify image with NN classifier
# save classified image and transition zone mask after classification takes place
 
saving classified LANDSAT/LC08/C02/T1_TOA/LC08_203024_20220113
 
saving trans

In [19]:
os.makedirs

<function os.makedirs(name, mode=511, exist_ok=False)>

In [26]:
output

{'L8': {'dates': [],
  'times': [],
  'veglines': [],
  'waterlines': [],
  'filename': [],
  'cloud_cover': [],
  'idx': [],
  'vthreshold': [],
  'wthreshold': [],
  'tideelev': []},
 'S2': {'dates': [],
  'times': [],
  'veglines': [],
  'waterlines': [],
  'filename': [],
  'cloud_cover': [],
  'idx': [],
  'vthreshold': [],
  'wthreshold': [],
  'tideelev': []}}